In [1]:
import pickle
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='/home/user/GITs/Zoomcamps/MLOps/mlops-material/4-Deployment/web-service-mlflow/artifacts-local/1', creation_time=1718785859986, experiment_id='1', last_update_time=1718785859986, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df["duration"] = df["lpep_dropoff_datetime"] - df["lpep_pickup_datetime"]
    df["duration"] = df["duration"].dt.total_seconds()

    df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

def prepare_dictionaries(df: pd.DataFrame):
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    dicts = df[categorical + numerical].to_dict(orient="records")
    return dicts

import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")
Obtaining the data from here:
- https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [6]:
df_train = read_dataframe("../data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2021-02.parquet")

target = "duration"
y_train = df_train[target]
y_val = df_val[target]

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

### Creating a scikit-learn pipeline & tracking it with mlflow

In [7]:
# Important: AWS credentials have to be set e.g. with aws configure

with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )
    
    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)
    
    rmse = MSE(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 16.53467808612593


/home/user/miniconda3/envs/mlops-zoomcamp-2024/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### Testing the connection with MLflow (to be used in predict.py)

In [8]:
from mlflow.tracking import MlflowClient

In [9]:
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"
RUN_ID = "5d5e054ed6294e40ac929d7a26f24e2b" # "10f4197008104ad183466cdb19e26c4e"  # AWS

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [10]:
path = client.download_artifacts(run_id=RUN_ID, path="dict_vectorizer.bin")

/tmp/ipykernel_13692/4226888799.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id=RUN_ID, path="dict_vectorizer.bin")


In [11]:
# Load the fitted dict vectorized
with open(path, "rb") as f_in:
    dv = pickle.load(f_in)

In [12]:
dv

DictVectorizer()